In [ ]:
import os

os.chdir("../..")

In [ ]:
%load_ext autoreload
%autoreload 2

from mars.models_training import data_loading
from mars.models_training.notebook_helpers import evaluate

import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
from mars.models_training import document_level_similarity_search


In [ ]:
X, y = data_loading.load_document_level_issues_dataset(data_loading.DocumenLevelDataset.ethics_ai_ethics, "laser")

In [ ]:
df_labels = pd.read_csv('data/labels_hagendorffEthicsAIEthics2020.csv', index_col=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Baseline

In [ ]:
X_max = list(map(max, X_test))

In [ ]:
evaluate(X_max, y_test)

In [ ]:
def bucketize(scores, bins):
    return np.stack([np.histogram(x, range=(-1, 1), bins=bins)[0] for x in scores])

In [ ]:
from sklearn.feature_selection import RFE,RFECV

In [ ]:
buckets_num = 250

model = document_level_similarity_search.model
model.fit(X_train, y_train)

In [ ]:
model = document_level_similarity_search.model
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
model2 = document_level_similarity_search.model_with_normalization
model2.fit(X_train, y_train)
y_pred = model2.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
buckets_num = 250

model3 =  document_level_similarity_search.model_with_normalization_rfe
model3.set_params(bucketize__kw_args = {'bins': buckets_num})
model3.fit(X_train, y_train)
y_pred = model3.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
param_grid = {
    "rfe__n_features_to_select": list(range(10,101,5)),
    "bucketize__kw_args":[{"bins":int(n)} for n in np.linspace(50,400,num=40)]
}

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


cv = RandomizedSearchCV(model3, param_grid, scoring="roc_auc",n_jobs=-1, verbose=1, n_iter=150)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
best_model = cv.best_estimator_
cv.best_score_, cv.best_params_

In [ ]:
y_pred = best_model.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
y_pred = best_model.predict_proba(X_test)[:, 1]
evaluate(y_pred, y_test)

In [ ]:
import pickle

pickle.dump(best_model, open("models/ethics_ai_ethics_laser_model.pkl", "wb"))

In [ ]:
import pickle

model = pickle.load(open('models/ethics_ai_ethics_laser_model.pkl', 'rb'))

In [ ]:
model